# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


# Importing Libraries

In [1]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [2]:
# Use this cell for any import statements you add

import shutil
from pathlib import Path

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [3]:
artists = {'lildicky':"https://www.azlyrics.com/l/lildicky.html",
           'lilwayne':"https://www.azlyrics.com/l/lilwayne.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

In [4]:
artists

{'lildicky': 'https://www.azlyrics.com/l/lildicky.html',
 'lilwayne': 'https://www.azlyrics.com/l/lilwayne.html'}

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: It is allowed because under user 008 (which is used to describe web crawlers) nothing is disallowed. In other words, web crawlers can scrape anything from the site.


In [5]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)
urls = []

for artist, artist_page in artists.items() :
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    soup = BeautifulSoup(r.text, 'html.parser')
    links = soup.find_all('a', href=True)
    for link in links:
        if '/lyrics/lil' in link['href']:
            urls.append(link.get('href'))
            lyrics_pages[artist].append(link.get('href'))


    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    

Let's make sure we have enough lyrics pages to scrape. 

In [6]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [7]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For lildicky we have 82.
The full pull will take for this artist will take 0.23 hours.
For lilwayne we have 742.
The full pull will take for this artist will take 2.06 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [8]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [9]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [10]:
soup = BeautifulSoup(requests.get("https://www.azlyrics.com/lyrics/lildicky/whitedude.html").content, "html.parser")
soup.select_one(".ringtone ~ div").get_text(
        strip=True, separator="\n"
    )

"[Hook: Lil Dicky]\nAnd y'ain't even want it with da boy\nCause the way I'm livin life, is a muthafuckin joy\nOn some grown man B.I.\nI could have been a girl\nOr any ethnicity up in the world\nBut I'm rollin with the top back\nI ain't gotta worry where the cops at\nI ain't gotta wear a fucking bra strap\nMe and the crew\nAre really doing everything that we like to\nMan it's a damn good day to be a white dude\n[Verse 1: Lil Dicky]\nShit, I'm living good up in this bitch\nDicky rollin round the city, blowin good up in that wip\nI ain't black or Dominican, not Hispanic or Indian\nSo imprisonment is not a predicament, I envision\nFor a white boy\nHollatcha muthafuckin kike, boy\nHigher than a muthafuckin kite, it's iight boi\nThey ain't suspicious of jews\nI'll show contrition it's cool\nSlap on the wrist, get ma daddy to choose\nOne of them top lawyers dat be rollin round in his crew\nI'm eating chop soy and bowlin on a carnival cruise\nDas what I do kid\nHappy that my name ain't stupid\

In [11]:
os.chdir("/Users/halledavis/Desktop/")

In [12]:
for artist in lyrics_pages :
    artistfolder = "lyrics/"+artist
    if os.path.isdir(artistfolder) : 
        shutil.rmtree(artistfolder)
    os.mkdir(artistfolder)

In [13]:
subs = 'lildicky'
lildickyurls = list(filter(lambda x: subs in x, urls))

In [14]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

os.chdir("/Users/halledavis/Desktop/lyrics/lildicky")

path = os.getcwd()

for x in lildickyurls :
    if total_pages == 22:
        break
    name0 = x.replace('/', '')
    name = name0.replace('.html', '')
    filename = name+".txt"
    if os.path.exists(filename):
        file = open(filename, "r+")
    else:
        file = open(filename, "w+")
    soup = BeautifulSoup(requests.get(url_stub+x).content, "html.parser")
    time.sleep(5 + 10*random.random())
    lyrics = soup.select_one(".ringtone ~ div").get_text(strip=True, separator="\n")
    title = name.replace('lyricslildicky', '')
    file.writelines(title+'\n'+'\n'+lyrics)
    time.sleep(5 + 10*random.random())
    total_pages += 1
    

In [15]:
subs = 'lilwayne'
lilwayneurls = list(filter(lambda x: subs in x, urls))

In [16]:
url_stub = "https://www.azlyrics.com" 

total_pages = 0 

os.chdir("/Users/halledavis/Desktop/lyrics/lilwayne")

path = os.getcwd()

for x in lilwayneurls :
    if total_pages == 22:
        break
    name0 = x.replace('/', '')
    name = name0.replace('.html', '')
    filename = name+".txt"
    if os.path.exists(filename):
        file = open(filename, "a+")
    else:
        file = open(filename, "w+")
    soup = BeautifulSoup(requests.get(url_stub+x).content, "html.parser")
    time.sleep(5 + 10*random.random())
    lyrics = soup.select_one(".ringtone ~ div").get_text(strip=True, separator="\n")
    title = name.replace('lyricslilwayne', '')
    file.writelines(title+'\n'+'\n'+lyrics)
    time.sleep(5 + 10*random.random())
    total_pages += 1
    

In [17]:
    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    # 2. Iterate over the lyrics pages
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
    # 4. Extract the title and lyrics from the page.
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist

In [18]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.24 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [19]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [20]:
os.chdir("/Users/halledavis/Desktop/")
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For lildicky we have 22 files.
For lildicky we have roughly 14667 words, 2753 are unique.
For lilwayne we have 22 files.
For lilwayne we have roughly 16840 words, 2054 are unique.
